# EDA 

In [ ]:
# Importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import seaborn as sns
sns.set()

Given our dataset located at [ChestX-ray8 dataset](https://arxiv.org/abs/1705.02315) we can explore the data and observe trends, summaries, and visualizations. 

In [7]:
patient_data = pd.read_csv("../Data_Entry_2017_v2020.csv")

In [13]:
patient_data.head(15)

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y]
0,00000001_000.png,Cardiomegaly,0,1,57,M,PA,2682,2749,0.143,0.143
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168
3,00000002_000.png,No Finding,0,2,80,M,PA,2500,2048,0.171,0.171
4,00000003_001.png,Hernia,0,3,74,F,PA,2500,2048,0.168,0.168
5,00000003_002.png,Hernia,1,3,75,F,PA,2048,2500,0.168,0.168
6,00000003_003.png,Hernia|Infiltration,2,3,76,F,PA,2698,2991,0.143,0.143
7,00000003_004.png,Hernia,3,3,77,F,PA,2500,2048,0.168,0.168
8,00000003_005.png,Hernia,4,3,78,F,PA,2686,2991,0.143,0.143
9,00000003_006.png,Hernia,5,3,79,F,PA,2992,2991,0.143,0.143


Our patient data totals to 112,120 total medical images for 30,805 unique patients. These images have been collected from patients spanning 1992 - 2015 from the clinical PACS database at NIHCC (National Institutes of Health Clinical Center) in which image labels are mined via NLP from radiology reports. The text-mined disease labels are expectedly >90% accuracy. Images are labeled as XXX_YYY.png, where XXX represents patient ID and YYY represents the CXR ID (or followup) of that patient.

We have a total of 14 of the most common thoracic pathologies:

Atelectasis, Cardiomegaly, Effusion, Infiltration, Mass, Nodule, Penumonia, Penumothorax, Consolidation, Edema, Emphysema, Fibrosis, Pleural Thickening, and Hernia.

Since pathologies can co-exist, we will incorporate multi-label classification.

Using our first patient (57M) as an example, we can see their first CXR (00000001_000.png) which is labeled with cardiomegaly. 

<img src="../images_001/00000001_000.png" alt="U-net Image" width="300" align="middle"/>
